In [2]:
import os
import openai

from dotenv import load_dotenv
load_dotenv() 

openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user", "content": "Hi my name is Earl, how are you?"}
    ],
    functions=[

        {
            "name": "translateToSpanish",
            "description": "Translate to spanish",
            "parameters": {
                'title': 'StepByStepAIResponse',
                'type': 'object',
                'properties': {'english': {'title': 'english', 'type': 'string'},
                               'spanish': {'title': 'spanish', 'type': 'string'}},
                'required': ['english', 'spanish']
            }
        }
    ],
    function_call={"name": "translateToSpanish"}
)

In [ ]:
print(res.choices[0]["message"]["function_call"]["arguments"])

first step is to get intent or http verb
extract the target
query the database for target
extract params
build the api call
confirmwith the user
call the api
display api call status

Here I can extract the HTTP VERB from the context, might need to adjust to work well with UKG's APIs

In [38]:
res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role":"assistant","content":"Only respond in HTTP Verbs such as GET, POST, PUT and DELETE"},
        {"role": "user", "content": "Who is John Snow?"}
    ],
    functions=[

        {
            "name": "promptToHTTPVerb",
            "description": "Extract the http verb from the string and return only a single http verb",
            "parameters": {
                'title': 'Single AI Response',
                'type': 'object',
                'properties': {'verb': {'title': 'verb', 'type': 'string'}},
                'required': ['verb']
            }
        }
    ],
    function_call={"name": "promptToHTTPVerb"}
)
print(res.choices[0]["message"]["function_call"]["arguments"])


{
  "verb": "GET"
}


In [7]:
def promptToHTTPVerb(prompt):
    return openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "assistant",
                "content": "Only respond in HTTP Verbs such as GET, POST, PUT and DELETE"},
            {"role": "user", "content": prompt}
        ],
        functions=[

            {
                "name": "promptToHTTPVerb",
                "description": "Extract the http verb from the string and return only a single http verb",
                "parameters": {
                    'title': 'Single AI Response',
                    'type': 'object',
                    'properties': {'verb': {'title': 'verb', 'type': 'string'}},
                    'required': ['verb']
                }
            }
        ],
        function_call={"name": "promptToHTTPVerb"}
    ).choices[0]["message"]["function_call"]["arguments"]


Here I can extract the subject of the users reply

In [ ]:
res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role":"assistant","content":"Only respond with the name of the subject of the message."},
        {"role": "user", "content": "My name is Kunta Kinte, Who is John Snow?"}
    ],
    functions=[
        {
            "name": "extractSubject",
            "description": "Extract the subject of a users reply",
            "parameters": {
                'title': 'Single AI Response',
                'type': 'object',
                'properties': {'subject': {'title': 'subject', 'type': 'string'}},
                'required': ['subject']
            }
        }
    ],
    function_call={"name": "extractSubject"}
)
print(res.choices[0]["message"]["function_call"]["arguments"])

Here I try to generate the params fo send a get request to the UKG API

In [ ]:
res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "system", "content": "Only respond with the name of the subject of the message."},
        {"role": "user", "content": "My name is Kunta Kinte, Who is John Snow?"}
    ],
    functions=[
        {
            "name": "generateParamsObject",
            "description": "Generate an object that stores the firstName and LastName",
            "parameters": {
                'title': 'Single AI Response',
                'type': 'object',
                'properties': {
                    'firstName': {'title': 'firstName', 'type': 'string'}, 
                    'lastName': {'title': 'lastName', 'type': 'string'}
                    },
                'required': ['firstName', 'lastName']
            }
        }
    ],
    function_call={"name": "generateParamsObject"}
)
print(res.choices[0]["message"]["function_call"]["arguments"])


In [ ]:
import requests

def get_employee_data(first_name, last_name):
    base_url = "https://api.ukg.com/employee"
    api_key = "YOUR_API_KEY"  # Replace with your actual API key

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    params = {
        "firstName": first_name,
        "lastName": last_name
    }

    try:
        response = requests.get(base_url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

# Example usage
employee_data = get_employee_data("John", "Snow")

if employee_data:
    # Process the employee data
    print("Employee Data:")
    print(f"Name: {employee_data['name']}")
    print(f"Department: {employee_data['department']}")
    # Add more fields as needed
else:
    print("No employee data found for the given name.")


In [62]:
res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role":"assistant","content":'''Only respond with 'true' or 'false' given a user message.
         if user response is nope, then respond false,if user response is no, then respond false,'''},
        {"role": "user", "content": "heck no"}
    ],
    functions=[
        {
            "name": "trueOrFalse",
            "description": "return 'true' if response is positive or affirmative, return 'false' if response is not positive or affirmative",
            "parameters": {
                'title': 'true/false response',
                'type': 'object',
                'properties': {'boolean': {'title': 'boolean', 'type': 'boolean'}},
                'required': ['boolean']
            }
        }
    ],
    function_call={"name": "trueOrFalse"}
)
print(res.choices[0]["message"]["function_call"]["arguments"])

{
  "boolean": false
}


In [8]:
chats = [
    {"role": "system", "content": "you are a helpful assistant"},
    {"role": "assistant", "content": "How may I help you"},
]


while True:

    prompt = input("What do you want")
    chats.append({"role": "user", "content": prompt})

    verb = promptToHTTPVerb(prompt)
    print(verb)

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=chats
    )

    chats.append({"role": "assistant", "content": completion.choices[0].message.content})
    print(chats)
    break


{
  "verb": "update"
}
[{'role': 'system', 'content': 'you are a helpful assistant'}, {'role': 'assistant', 'content': 'How may I help you'}, {'role': 'user', 'content': 'update jhon showa profile'}, {'role': 'assistant', 'content': "I'm sorry, but as an AI assistant, I do not have the capability to update someone's profile. However, if you need guidance on how to update a profile on a specific platform or website, I can certainly assist you with that. Please provide more details on the specific platform or website you are referring to."}]
